<a href="https://colab.research.google.com/github/merucode/RL/blob/36-Study-Doc-Custom_grid_world_with_render/02_Reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone -b 36-Study-Doc-Custom_grid_world_with_render https://github.com/merucode/RL.git
!cd RL && mv * ../

fatal: destination path 'RL' already exists and is not an empty directory.
mv: cannot stat '*': No such file or directory


In [23]:
!pip install gym==0.25.2
!pip install -e gym-examples

Obtaining file:///content/gym-examples
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gym-examples
    Found existing installation: gym-examples 0.0.1
    Uninstalling gym-examples-0.0.1:
      Successfully uninstalled gym-examples-0.0.1
  Running setup.py develop for gym-examples


### NOTE: After inistall gym-examples, Restart runtime

In [1]:
import time
import matplotlib.pyplot as plt
import gym
import numpy as np

In [2]:
env = gym.make("gym_examples:gym_examples/GridWorld-v0")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# single env
print(f'observation space : {env.observation_space}')
print(f'action space      : {env.action_space}')
print(f'action shape      : {env.action_space.n}')

observation space : Dict('agent': Box(0, 4, (2,), int64), 'target': Box(0, 4, (2,), int64))
action space      : Discrete(4)
action shape      : 4


In [9]:
obs = env.reset()
print(obs)

obs = np.hstack((obs['agent'], obs['target']))
print(obs)

{'agent': array([1, 2]), 'target': array([2, 2])}
[1 2 2 2]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
action = env.action_space.sample()
env.step(0)

({'agent': array([3, 4]), 'target': array([1, 2])},
 -0.01,
 False,
 {'distance': 4.0})

# Reinforce model

In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np

#Device
device = device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 4)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x

    def put_data(self, item):
        self.data.append(item)

    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R
            loss.backward()
        self.optimizer.step()
        self.data = []

In [24]:
def main():
    env = gym.make("gym_examples:gym_examples/GridWorld-v0")
    pi = Policy()
    score = 0.0
    max_score = 0.0
    print_interval = 100

    pi.to(device)

    for n_epi in range(3000):
        obs = env.reset()
        s = np.hstack((obs['agent'], obs['target']))
        done = False

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample()
            obs_next, r, done, info = env.step(a.item())
            s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
            pi.put_data((r,prob[a]))
            s = s_prime
            score += r

        pi.train_net()

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            if score > max_score:
                torch.save(pi.state_dict(), 'model_weights.pth')
                print(f"# max score : {score/print_interval}, and save model_weights.pth")
                max_score = score
            score = 0.0

    env.close()


In [25]:
if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


# of episode :100, avg score : 0.3661999999999984
# max score : 0.3661999999999984, and save model_weights.pth
# of episode :200, avg score : 0.3965999999999997
# max score : 0.3965999999999997, and save model_weights.pth
# of episode :300, avg score : 0.48129999999999656
# max score : 0.48129999999999656, and save model_weights.pth
# of episode :400, avg score : 0.6883000000000034
# max score : 0.6883000000000034, and save model_weights.pth
# of episode :500, avg score : 0.6175000000000065
# of episode :600, avg score : 0.750399999999995
# max score : 0.750399999999995, and save model_weights.pth
# of episode :700, avg score : 0.7416000000000024
# of episode :800, avg score : 0.8284999999999808
# max score : 0.8284999999999808, and save model_weights.pth
# of episode :900, avg score : 0.7963999999999827
# of episode :1000, avg score : 0.8370999999999943
# max score : 0.8370999999999943, and save model_weights.pth
# of episode :1100, avg score : 0.8228999999999987
# of episode :1200, a

# Render

In [26]:
#env = gym.make('CartPole-v1')


# define a trigger function, return True to start recording a new video:
# video_every=1
# episode_trigger=lambda episode_id: ((episode_id)%video_every==0)
# decorate env with a video recorder:

def model_render(env_name):
    env = gym.make(env_name)
    episode_trigger=lambda e: True  # all Episode Recode

    env = gym.wrappers.RecordVideo(env, "./video", episode_trigger=episode_trigger, video_length=100000)

    for episode in range(10):
        obs = env.reset()
        s = np.hstack((obs['agent'], obs['target']))
        done = False
        score = 0

        pi = Policy()
        pi.load_state_dict(torch.load('model_weights.pth'))
        pi.eval()

        pi.to(device)

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample()
            obs_next, r, done, info = env.step(a.item())
            s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
            done_mask = 0.0 if done else 1.0
            s = s_prime
            score += r

        print(f"# of episode {episode}:, avg score : {score}")
    env.close()

In [27]:
model_render("gym_examples:gym_examples/GridWorld-v0")

/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment gym_examples/GridWorld-v0 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


# of episode 0:, avg score : 0.98
# of episode 1:, avg score : 0.97
# of episode 2:, avg score : 0.99
# of episode 3:, avg score : 0.7999999999999999
# of episode 4:, avg score : 0.91
# of episode 5:, avg score : 0.94
# of episode 6:, avg score : 0.84
# of episode 7:, avg score : 0.86
# of episode 8:, avg score : 0.99
# of episode 9:, avg score : 0.95


In [28]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('./video/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=500 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)